## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-01-22-46-49 +0000,nyt,"Live Updates: After a Weak Jobs Report, Trump ...",https://www.nytimes.com/live/2025/07/31/busine...
1,2025-08-01-22-46-47 +0000,nyt,"U.S. Hiring Slowed in July, Adding 73,000 Jobs",https://www.nytimes.com/2025/08/01/us/politics...
2,2025-08-01-22-41-49 +0000,wapo,Manhunt underway for Montana shooting suspect ...,https://www.washingtonpost.com/nation/2025/08/...
3,2025-08-01-22-39-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...
4,2025-08-01-22-35-00 +0000,wsj,Manhunt Under Way After Four Killed at Montana...,https://www.wsj.com/us-news/four-people-killed...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2302/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,83
203,tariffs,22
43,new,19
21,tariff,13
295,russian,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
6,2025-08-01-22-33-00 +0000,wsj,President Trump fired the top Bureau of Labor ...,https://www.wsj.com/politics/policy/trump-orde...,169
301,2025-07-31-23-55-00 +0000,wsj,President Trump is building a Mar-a-Lago style...,https://www.wsj.com/politics/policy/trump-ball...,143
278,2025-08-01-02-34-00 +0000,wsj,Lawmakers have introduced bills targeting Chin...,https://www.wsj.com/politics/policy/congress-l...,143
18,2025-08-01-22-08-04 +0000,nypost,Trump fires Biden-appointed Bureau of Labor St...,https://nypost.com/2025/08/01/us-news/trump-fi...,143
44,2025-08-01-21-29-00 +0000,wsj,Stock Market Today: Dow Falls on Tariff Blitz;...,https://www.wsj.com/livecoverage/jobs-report-t...,142


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
6,169,2025-08-01-22-33-00 +0000,wsj,President Trump fired the top Bureau of Labor ...,https://www.wsj.com/politics/policy/trump-orde...
10,56,2025-08-01-22-27-28 +0000,nypost,Trump admin fires back at claims Clinton plan ...,https://nypost.com/2025/08/01/us-news/trump-ad...
62,55,2025-08-01-20-59-00 +0000,wsj,"Ghislaine Maxwell, the former associate of con...",https://www.wsj.com/politics/policy/epstein-as...
26,52,2025-08-01-21-57-00 +0000,wsj,The Trump administration is blocking funding f...,https://www.wsj.com/politics/policy/trump-cdc-...
166,50,2025-08-01-15-34-42 +0000,nypost,"Dow drops over 500 points as Trump tariffs, jo...",https://nypost.com/2025/08/01/business/dow-dro...
161,50,2025-08-01-15-57-21 +0000,cbc,Viable trade deal wasn't on the table ahead of...,https://www.cbc.ca/news/politics/reaction-trum...
282,46,2025-08-01-02-17-15 +0000,nypost,Vile footage shows man slap woman for refusing...,https://nypost.com/2025/07/31/world-news/vile-...
211,45,2025-08-01-11-25-01 +0000,nypost,Russian attack on Kyiv leaves at least 28 dead...,https://nypost.com/2025/08/01/world-news/russi...
293,44,2025-08-01-00-49-00 +0000,wsj,President Trump says he has averted a half-doz...,https://www.wsj.com/politics/policy/trumps-vow...
132,43,2025-08-01-17-58-15 +0000,nypost,"Anti-Israel NYC Councilwoman Tiffany Caban, do...",https://nypost.com/2025/08/01/us-news/proteste...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
